In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [3]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [4]:
from torch.utils.data import TensorDataset

In [5]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [6]:
from torch.utils.data import DataLoader

In [7]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [8]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 135.,  57.],
        [ 73.,  67.,  43.],
        [ 73.,  66.,  44.],
        [103.,  43.,  36.],
        [ 69.,  96.,  70.]])
tensor([[118., 134.],
        [ 56.,  70.],
        [ 57.,  69.],
        [ 20.,  38.],
        [103., 119.]])


In [9]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.4167,  0.0257, -0.3934],
        [ 0.4714, -0.1354,  0.5077]], requires_grad=True)
Parameter containing:
tensor([-5.1800e-01, -1.0502e-04], requires_grad=True)


In [10]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4167,  0.0257, -0.3934],
         [ 0.4714, -0.1354,  0.5077]], requires_grad=True),
 Parameter containing:
 tensor([-5.1800e-01, -1.0502e-04], requires_grad=True)]

In [11]:
preds = model(inputs)
preds

tensor([[14.7088, 47.1694],
        [14.4886, 63.4722],
        [16.3652, 52.3104],
        [28.5357, 61.0446],
        [ 3.1670, 55.0638],
        [15.0998, 47.7762],
        [14.0695, 64.1153],
        [16.3885, 53.2895],
        [28.1447, 60.4377],
        [ 2.3569, 55.1001],
        [14.2897, 47.8125],
        [14.8796, 64.0790],
        [16.7842, 51.6673],
        [29.3458, 61.0083],
        [ 2.7760, 54.4570]], grad_fn=<AddmmBackward>)

In [12]:
import torch.nn.functional as F

In [13]:
loss_fn = F.mse_loss

In [14]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(3965.9604, grad_fn=<MseLossBackward>)


In [15]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [16]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [17]:
fit(100, model, loss_fn, opt, train_dl)

C:\Users\Deeksha\anaconda3\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Epoch [10/100], Loss: 802.5221
Epoch [20/100], Loss: 830.5322
Epoch [30/100], Loss: 334.1677
Epoch [40/100], Loss: 442.1022
Epoch [50/100], Loss: 159.1303
Epoch [60/100], Loss: 159.3002
Epoch [70/100], Loss: 70.8389
Epoch [80/100], Loss: 98.7890
Epoch [90/100], Loss: 60.0981
Epoch [100/100], Loss: 32.9664


In [18]:
preds = model(inputs)
preds

tensor([[ 58.6411,  71.9011],
        [ 78.5640, 100.7698],
        [122.6083, 130.1303],
        [ 31.2815,  46.2070],
        [ 89.6727, 113.9495],
        [ 57.5529,  71.0468],
        [ 77.7046, 100.8022],
        [122.5793, 130.7606],
        [ 32.3697,  47.0613],
        [ 89.9014, 114.8362],
        [ 57.7817,  71.9335],
        [ 77.4758,  99.9155],
        [123.4677, 130.0978],
        [ 31.0527,  45.3203],
        [ 90.7609, 114.8037]], grad_fn=<AddmmBackward>)

In [19]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])